# Follow up 10/01 meeting
## For Parag: 
Domain-level predictions 
  - for training and testing dataset
     - note: didn't save training predictions or model objects so inly test for now. hand over to parag and confirm he needs the training enough to justify the work
  - in big query
     - upload **tld_reseller_compare_predictions.csv**
  - calc: predicted renewal rate vs, actual renewal rate at tld-reg level
     - **tld_reseller_compare_rates.csv**
     - **tld_reseller_compare_sums.csv**

In [1]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [2]:
# install.packages("pkgcond")

In [3]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
library(rPython)
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)
library(dominanceanalysis)


getwd()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: grid

Loading required package: libcoin

Loading required package: mvtnorm

Loading required package: lattice

Loading required package: ggplot2

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:sta

[1] "/home/jupyter/local/Domains_202003/scripts/phaseII_06"

In [4]:

source('../orig/functions.R')

source('../orig/functions_models.R')

source('../phaseII_03_forest/functions_eval.R')


source('../phaseII_03_forest/load_prep_data_expiry.R')

# expiry_train_prepped_2_1 (list, less df's w/ 0 obs)
# expiry_test_prepped_2_1  (list, less df's w/ 0 obs)
# expiry_train_df_1 (above, rbound)
# expiry_test_df_1 (above, rbound)
# expiry_train_df_sub (subset vars)                                        
# expiry_test_df_sub (subset vars)

## Load PREPed tld_reseller compare

In [8]:
predictions_df <- read.csv("../../data/tld_reseller_compare_predictions.csv")
metrics_df <- read.csv("../../data/tld_reseller_compare_metrics.csv")

dim(predictions_df)
head(predictions_df)
dim(metrics_df)
head(metrics_df)

[1] 297100     12

,actual,pred_df_seg2_glm,pred_df_seg_glm,pred_df_agg_glm,pred_df_seg2_rf,pred_df_seg_rf,pred_df_agg_rf,tld_registrar_index,reseller,n,domain_id,domain
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<int>,<int>,<fct>
1,Not Renewd,0.6470992,0.558,0.419,0.4765771,0.4545657,0.5386137,sitegmo,gmo,106569,44295183,kagen.site
2,Renewed,0.6762222,0.567,0.397,0.5410605,0.5769973,0.6091513,sitegmo,gmo,106569,44573611,designlab.site
3,Renewed,0.9098495,0.835,0.410,0.7018108,0.6343404,0.6886680,sitegmo,gmo,106569,45304858,hokatu-blog.site
4,Renewed,0.6029350,0.515,0.402,0.2414726,0.3999603,0.3464197,sitegmo,gmo,106569,46235129,suzuya.site
5,Not Renewd,0.9311321,0.863,0.417,0.7749462,0.7039504,0.6711740,sitegmo,gmo,106569,46276970,wins-company.site
6,Renewed,0.9347869,0.861,0.391,0.6376848,0.6566740,0.6717237,sitegmo,gmo,106569,47809960,yamatoku-company.site


[1] 1723   15

,tld_registrar_index,reseller,n,l10_seg2_glm,l10_seg_glm,l10_agg_glm,l10_seg2_rf,l10_seg_rf,l10_agg_rf,auc_seg2_glm,auc_seg_glm,auc_agg_glm,auc_seg2_rf,auc_seg_rf,auc_agg_rf
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,sitegmo,gmo,106569,8.204334,8.126935,6.408669,8.777090,8.761610,8.591331,0.8958204,0.8825077,0.8729102,0.9345201,0.9348297,0.9329721
2,funalibaba,alibaba,72690,4.883721,5.255814,5.023256,5.813953,6.000000,6.186047,0.8262791,0.8430233,0.8518605,0.8769767,0.8779070,0.8793023
3,sitenamecheap,namecheap,53536,2.298025,1.921005,1.885099,2.280072,2.028725,2.046679,0.6862657,0.6196589,0.6338420,0.6654399,0.6792639,0.6893178
4,pwnamecheap,namecheap,50470,3.478261,3.675889,3.675889,3.438735,4.071146,4.308300,0.7274704,0.6551383,0.6361660,0.7167984,0.7405138,0.7507905
5,onlinego daddy,go daddy,42162,2.850995,2.883271,2.474449,2.598171,2.727273,2.646584,0.6506455,0.6493007,0.6381119,0.6095481,0.6266541,0.6249328
6,sitegandi sas,gandi sas,35309,8.243243,8.445946,7.094595,4.932432,8.445946,8.581081,0.9202703,0.9195946,0.9189189,0.7162162,0.9344595,0.9358108


In [26]:
sum(predictions_df$actual=='Renewed', na.rm=TRUE)/nrow(predictions_df)
sum(predictions_df$actual=='Renewed', na.rm=TRUE)/(nrow(predictions_df)-sum(is.na(predictions_df$actual)))

[1] 0.09124201

[1] 0.09136224

## tld-reg predicted vs. actual rate summary

In [34]:
options(scipen=999)
pred50_df <- predictions_df %>% 
  group_by(tld_registrar_index) %>% 
  summarise(
            actual_rate = sum( actual == 'Renewed' )/n(),
            pred50_seg2_glm = sum( pred_df_seg2_glm > 0.5 )/n(),
            pred50_seg_glm = sum( pred_df_seg_glm > 0.5 )/n(),
            pred50_agg_glm = sum( pred_df_agg_glm > 0.5 )/n(),
            pred50_seg2_rf = sum( pred_df_seg2_rf > 0.5 )/n(),
            pred50_seg_rf = sum( pred_df_seg_rf > 0.5 )/n(),
            pred50_agg_rf = sum( pred_df_agg_rf > 0.5 )/n(),
            count = n()) %>% 
  arrange(desc(count)) 
head(pred50_df)

`summarise()` ungrouping output (override with `.groups` argument)



tld_registrar_index,actual_rate,pred50_seg2_glm,pred50_seg_glm,pred50_agg_glm,pred50_seg2_rf,pred50_seg_rf,pred50_agg_rf,count
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
sitegmo,0.02445210,0.00166546803,0.0011355464,0.000000000,0.0008705856,0.0004163670,0.00045421855,26419
funalibaba,0.01180800,0.00005492091,0.0000000000,0.000000000,0.0001098418,0.0000000000,0.00005492091,18208
sitenamecheap,0.04168538,0.00014967819,0.0000000000,0.000000000,0.0003741955,0.0000748391,0.00014967819,13362
pwnamecheap,0.02003008,0.00023751089,0.0002375109,0.000000000,0.0006333624,0.0005541921,0.00031668118,12631
onlinego daddy,0.17398222,0.04445484324,0.0456715021,0.001216659,0.0208703790,0.0149742630,0.01422554984,10685
sitegandi sas,0.01697442,0.00000000000,0.0002293841,0.000000000,0.0006881523,0.0000000000,0.00000000000,8719


In [37]:
options(scipen=999)
sum_df <- predictions_df %>% 
  group_by(tld_registrar_index) %>% 
  summarise(
            actual_count = sum( actual == 'Renewed' ),
            sum_seg2_glm = round(sum( pred_df_seg2_glm ),0),
            sum_seg_glm = round(sum( pred_df_seg_glm ),0),
            sum_agg_glm = round(sum( pred_df_agg_glm ),0),
            sum_seg2_rf = round(sum( pred_df_seg2_rf ),0),
            sum_seg_rf = round(sum( pred_df_seg_rf ),0),
            sum_agg_rf = round(sum( pred_df_agg_rf ),0),
            count = n()) %>% 
  arrange(desc(count)) 
head(sum_df)

`summarise()` ungrouping output (override with `.groups` argument)



tld_registrar_index,actual_count,sum_seg2_glm,sum_seg_glm,sum_agg_glm,sum_seg2_rf,sum_seg_rf,sum_agg_rf,count
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
sitegmo,646,1172,1244,1480,1302,1202,1198,26419
funalibaba,215,451,580,527,484,519,526,18208
sitenamecheap,557,709,933,1041,965,952,911,13362
pwnamecheap,253,497,564,569,837,724,666,12631
onlinego daddy,1859,2581,2467,2381,1993,1891,1953,10685
sitegandi sas,148,165,284,180,1133,338,263,8719


In [38]:
write.csv(pred50_df, "../../data/output/tld_reseller_compare_rates.csv", row.names = FALSE)
write.csv(sum_df, "../../data/output/tld_reseller_compare_sums.csv", row.names = FALSE)

In [39]:
system("gsutil cp /home/jupyter/local/Domains_202003/data/output/* gs://data_outputt/output/")
system("mv /home/jupyter/local/Domains_202003/data/output/* /home/jupyter/local/Domains_202003/data/")